In [ ]:
# Requirements - uncomment as needed
#!pip install matplotlib numpy scipy phyre

# Generating the dataset

The requirement for the dataset here: 

- only using the template that contains only two balls on the scene;

- red ball must be in the free-fall without any collisions other than with the floor;

- All balls are recorded.

## Playground 1

For now these playgrounds are just moved from another notebook. NEEDS CHANGES!

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import re

import phyre

In [ ]:
# Choosing a setup where only one ball is needed
eval_setup = 'ball_cross_template'

# Getting 10 folds of data from the simulator
fold_ids = range(1,11)

# Let's only check the first fold for now
fold_id = 0
train_tasks, dev_tasks, test_tasks = phyre.get_fold(eval_setup, fold_id)
print(f'Size of resulting splits for fold {fold_id}:\n train:', len(train_tasks), '\n dev:',
      len(dev_tasks), '\n test:', len(test_tasks))

In [ ]:
# combining all tasks in one list
tasks = list(train_tasks + dev_tasks + test_tasks)
task_filter = re.compile("00000:*")
tasks = list(filter(task_filter.match, tasks))
tasks

In [ ]:
# Getting action tier for our tasks - a single ball
action_tier = phyre.eval_setup_to_action_tier(eval_setup)
print('Action tier for', eval_setup, 'is', action_tier)

In [ ]:
# Create the simulator from the tasks and tier.
simulator = phyre.initialize_simulator(tasks, action_tier)

In [ ]:
# getting a 1000 actions from a simulator
# it uniformly samples actions skipping invalid ones
# Action dimensions: 3 (x, y, radius)

actions = simulator.build_discrete_action_space(max_actions=1000)

In [ ]:
# Temp - TODO to be replaced with a loop
task_index = 0
action = actions[0]

simulation = simulator.simulate_action(task_index, action, need_images=True, need_featurized_objects=True)

In [ ]:
# Displaying the simulation
print('Number of observations returned by simulator:', len(simulation.images))

num_across = 5
height = int(math.ceil(len(simulation.images) / num_across))
fig, axs = plt.subplots(height, num_across, figsize=(20, 15))
fig.tight_layout()
plt.subplots_adjust(hspace=0.2, wspace=0.2)

# We can visualize the simulation at each timestep.
for i, (ax, image) in enumerate(zip(axs.flatten(), simulation.images)):
    # Convert the simulation observation to images.
    img = phyre.observations_to_float_rgb(image)
    ax.imshow(img)
    ax.title.set_text(f'Timestep {i}')
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])

In [ ]:
print
print(simulation.featurized_objects.features)

In [ ]:
def is_red_ball_in_free_fall(features):
    return False not in [features[0][-1][0] == features[frame_id][-1][0] for frame_id in range(len(features))]

In [ ]:
is_red_ball_in_free_fall(simulation.featurized_objects.features)

## Playground + Generation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import re
import itertools

import phyre

In [ ]:
# Choosing a setup where only one ball is needed
eval_setup = 'ball_cross_template'


# We only need one fold as we mix all the data together anyway
fold_id = 0

train_tasks, dev_tasks, test_tasks = phyre.get_fold(eval_setup, fold_id)
tasks = list(train_tasks + dev_tasks + test_tasks)

print('Total tasks: ', len(tasks))

In [ ]:
# Filtering tasks to only include a simple two-ball template. The template key: '00000:xxx'

task_filter = re.compile("00000:*")
tasks = list(filter(task_filter.match, tasks))

print('Tasks after filtering: ', len(tasks))

In [ ]:
# Getting action tier for our tasks - a single ball
action_tier = phyre.eval_setup_to_action_tier(eval_setup)
print('Action tier for', eval_setup, 'is', action_tier)

In [ ]:
# Create the simulator from the tasks and tier.
simulator = phyre.initialize_simulator(tasks, action_tier)

In [ ]:
# getting a 1000 actions from a simulator
# it uniformly samples actions skipping invalid ones
# Action dimensions: 3 (x, y, radius) - represent coordinates and size of the red ball

actions = simulator.build_discrete_action_space(max_actions=1000)

In [ ]:
# Defining a function to check if the red ball is in the free fall throughout the simulation
def is_red_ball_in_free_fall(simulation):
    features = simulation.featurized_objects.features
    return False not in [features[0][-1][0] == features[frame_id][-1][0] for frame_id in range(len(features))]

In [ ]:
# Getting only the coordinates of the red ball
def get_red_ball_data(simulation):
    features = simulation.featurized_objects.features
    data = []
    for frame_id in range(len(features)):
        data.append([features[frame_id][-1][0], features[frame_id][-1][1]])
    return data

In [ ]:
# DO NOT INCLUDE IT IN THE SCRIPT
# Checking what is in one of the files that I saved
np.load('data-free-fall/task-0-action-0.npy')